# Naive Bayes - Trabalho

## Questão 1

Implemente um classifacor Naive Bayes para o problema de predizer a qualidade de um carro. Para este fim, utilizaremos um conjunto de dados referente a qualidade de carros, disponível no [UCI](https://archive.ics.uci.edu/ml/datasets/car+evaluation). Este dataset de carros possui as seguintes features e classe:

** Attributos **
1. buying: vhigh, high, med, low
2. maint: vhigh, high, med, low
3. doors: 2, 3, 4, 5, more
4. persons: 2, 4, more
5. lug_boot: small, med, big
6. safety: low, med, high

** Classes **
1. unacc, acc, good, vgood

## Questão 2
Crie uma versão de sua implementação usando as funções disponíveis na biblioteca SciKitLearn para o Naive Bayes ([veja aqui](http://scikit-learn.org/stable/modules/naive_bayes.html)) 

## Questão 3

Analise a acurácia dos dois algoritmos e discuta a sua solução.

In [ ]:
import pandas as pd
import numpy as np
import csv
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import random

In [ ]:
def loadCsv(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [str(x) for x in dataset[i]]
    return dataset

def splitDataset(dataset, splitRatio):
    trainSize = int(len(dataset) * splitRatio)
    trainSet = []
    copy = list(dataset)
    while len(trainSet) < trainSize:
        index = random.randrange(len(copy))
        trainSet.append(copy.pop(index))
    return [trainSet, copy]

def separateByClass(dataset):
    separated = {}
    for _,row in dataset.iterrows():
        vector = row
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

def mean(numbers):
    return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    avg = mean(numbers)
    variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
    return math.sqrt(variance)

def summarize(dataset):
    summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
    del summaries[-1]
    return summaries

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.items():
        summaries[classValue] = summarize(instances)
    return summaries

def calculateProbability(x, mean, stdev):
    exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
    return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
    probabilities = {}
    for classValue, classSummaries in summaries.items():
        probabilities[classValue] = 1
        for i in range(len(classSummaries)):
            mean, stdev = classSummaries[i]
            if stdev == 0.0:
                stdev = 10000.0
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

def getPredictions(summaries, testSet):
    predictions = []
    for _,row in testSet.iterrows():
        result = predict(summaries, row)
        predictions.append(result)
    return predictions

def getAccuracy(testSet, predictions):
    correct = 0
    for i in range(len(testSet)):
        if testSet.iloc[i] == predictions[i]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [ ]:
data=loadCsv('carData.csv')


In [ ]:
def encode(dataset):
    encoded_copy = dataset
    feature_list = []
    for i in range(len(dataset[0])):
        listE = []
        feature_list.append(listE)
        
    for i in range(len(dataset[0])):
        for j in dataset:
            if(j[i] not in feature_list[i]):
                feature_list[i].append(j[i])
                 
    for i in range(len(encoded_copy[0])):
        for j in encoded_copy:
            if(j[i] in feature_list[i]):
                j[i] = feature_list[i].index(j[i])
            
    return encoded_copy
    

In [ ]:
d = encode(data)
train, test = splitDataset(d, 0.7)

In [ ]:

summaries = summarizeByClass(train)
predictions = getPredictions(summaries, test)
accuracy = getAccuracy(test, predictions)

accuracy

In [ ]:
datas = pd.read_csv('carData.csv')
datas.head()

In [ ]:
datas = pd.read_csv('carData.csv', names=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'classe'])

datas['buying'] = LabelEncoder().fit(datas.buying).transform(datas.buying)
datas['maint'] = LabelEncoder().fit(datas.maint).transform(datas.maint)
datas['doors'] = LabelEncoder().fit(datas.doors).transform(datas.doors)
datas['persons'] = LabelEncoder().fit(datas.persons).transform(datas.persons)
datas['lug_boot'] = LabelEncoder().fit(datas.lug_boot).transform(datas.lug_boot)
datas['safety'] = LabelEncoder().fit(datas.safety).transform(datas.safety)
datas['classe'] = LabelEncoder().fit(datas.classe).transform(datas.classe)

In [ ]:
datas.head()

In [ ]:
pvt=['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'classe']
x = MinMaxScaler().fit_transform(datas[pvt])

y = datas.classe.values

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [ ]:
gnb = GaussianNB()

In [ ]:
y_pred = gnb.fit(x_train, y_train).predict(x_test)
accuracy_score(y_test, y_pred)